In [ ]:
import pinecone
from text2vec import SentenceModel

In [118]:
pinecone.init(api_key='54dda629-c12f-4633-bb8e-d139a55a77f9', environment='us-west1-gcp-free')
index = pinecone.Index('student')

In [ ]:
import pandas as pd
import tiktoken

In [ ]:
df = pd.read_csv('data/test.csv')

In [ ]:
df.head()

In [ ]:
def tokenlizer_csv(file_root: str) -> pd.DataFrame:
    embedding_encoding = "cl100k_base"
    encoding = tiktoken.get_encoding(embedding_encoding)

    df = pd.read_csv(file_root)
    df = df[["question", "answer"]]
    df["summarized"] = ("question: " + df.question.str.strip() + "; answer: " + df.answer.str.strip())

    df["tokens"] = df.summarized.apply(lambda x: len(encoding.encode(x)))
    return df

In [ ]:
df = tokenlizer_csv('data/test.csv')

In [ ]:
def get_text_embedding(text):
    model = SentenceModel('./text2vec-base-chinese')
    embeddings = model.encode(text)
    return embeddings

In [ ]:
def get_df_embedding(df):
    return {idx: get_text_embedding(r.summarized) for idx, r in df.iterrows()}

In [ ]:
df_embedding = get_df_embedding(df)

In [ ]:
df_embedding

In [ ]:
df_embedding[0].tolist()

In [ ]:
vectors = []
count = 0
for embed_index in df_embedding:
    var = {
        'id': str(count),
        'values': df_embedding[embed_index].tolist()
    }
    vectors.append(var)
    count+=1


In [ ]:
vectors

In [ ]:
upsert_response = index.upsert(
    vectors=vectors,
)

In [ ]:
query = "早上要跑步吗"

In [ ]:
xq = get_text_embedding(query).tolist()


In [ ]:
xc = index.query(xq, top_k=5, include_metadata=True)

In [90]:
xc

{'matches': [{'id': '3', 'score': 0.611040115, 'values': []},
             {'id': '0', 'score': 0.240501195, 'values': []},
             {'id': '2', 'score': 0.231632277, 'values': []},
             {'id': '1', 'score': 0.163174063, 'values': []}],
 'namespace': ''}

In [101]:
[value["id"] for value in xc["matches"]]

['3', '0', '2', '1']

In [112]:
print(df.iloc[3].to_dict())

{'question': '有晨跑吗？', 'answer': '没有', 'summarized': 'question: 有晨跑吗？; answer: 没有', 'tokens': 18}


In [113]:
type(df)

pandas.core.frame.DataFrame

In [88]:
index.delete(ids=["3"])

{}

In [120]:
index.describe_index_stats()

{'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 4}, 'we': {'vector_count': 39}},
 'total_vector_count': 43}